After trying on the fpga, we have a file to read and check through

In [ ]:
import numpy as np

def to_int(binary_str):
    n = len(binary_str)
    value = int(binary_str, 2)
    if binary_str[0] == '1':
        value -= (1 << n)
    return value

count = 0

with open("log.txt", "r") as f:
    lines = f.readlines()
    for line_num, line in enumerate(lines):
        line = line.strip()

        num1_end = line.index(" / ")
        num1 = to_int(line[:num1_end])

        num2_start = line.index(" / ") + 3
        num2_end = line.index(" = ")
        num2 = to_int(line[num2_start:num2_end])

        result_start = line.index(" = ") + 3
        result_end = line.index(" R ")
        result = to_int(line[result_start:result_end])

        rem_start = line.index(" R ") + 3
        rem = to_int(line[rem_start:])

        result_exp = int(np.trunc(num1 / num2))
        result_exp = (result_exp & 0xFFFF_FFFF)

        rem_exp = (num1 - result_exp * num2)
        rem_exp = (rem_exp & 0xFFFF_FFFF)

        result_exp = (result_exp & 0xFFFF_FFFF) - (1 << 32) if result_exp >= (1 << 31) else result_exp
        rem_exp = (rem_exp & 0xFFFF_FFFF) - (1 << 32) if rem_exp >= (1 << 31) else rem_exp

        assert result == result_exp, f"Failed for line {line_num}: {num1} / {num2} -> {result} R {rem} != {result_exp} R {rem_exp}"
        assert rem == rem_exp, f"Failed for line {line_num}: {num1} / {num2} -> {result} R {rem} != {result_exp} R {rem_exp}"

        count += 1

print(f"All {count} tests passed!")

AssertionError: Failed for line 0: -2147461593 / 1073752900 -> -1 R -1073708693 != -1 R 44207